In [1]:
import pandas 
import json

In [82]:
data = json.load(open('exit_pole_data.json'))
data = data['props']['pageProps']['data']
df = pandas.DataFrame(data)
df = df.query("noData == False")

df['previousWinner'] = df['previousWinner'].apply(lambda x: x.get('name'))
#df['forecasts'] = df['forecasts'].apply(lambda x: x[0].get('party').get('name') if len(x) > 0 else None)


def get_forecasted(forecasts):
    if type(forecasts) != list or len(forecasts) == 0:
        return False
    if forecasts[0]['party']['name'] != 'Too close to call':
        return forecasts[0]['party']['name']
    elif len(forecasts) > 1:
        return forecasts[1]['party']['name']
    else:
        return None


df['forecast'] = df['forecasts'].apply(get_forecasted)

df = df[['name', 'forecast', 'previousWinner']]
df['change'] = df['forecast'] != df['previousWinner']
df['name'] = df.name.str.replace('&', 'and')

df[df.name.str.contains("Cheshire")]

h = json.load(open('hexes_long.geojson'))
h = h.get('features')
names = [x.get('properties').get('BCName') for x in h]
codes = [x.get('properties').get('GSScode') for x in h]
regions = [x.get('properties').get('CTR_REG') for x in h]
hex_df = pandas.DataFrame({'name': names, 'code': codes, 'region': regions})

lookups = {
  "Cheshire Mid": "Mid Cheshire",
  "Bedfordshire Mid": "Mid Bedfordshire",
  "Dorset North": "North Dorset",
  "Shropshire South": "South Shropshire",
  "Ayrshire Central": "Central Ayrshire",
  "Cambridgeshire South": "South Cambridgeshire",
  "Surrey East": "East Surrey",
  "Leicestershire Mid": "Mid Leicestershire",
  "Ayrshire North and Arran": "North Ayrshire and Arran",
  "Cotswolds North": "North Cotswolds",
  "Worcestershire West": "West Worcestershire",
  "Shropshire North": "North Shropshire",
  "Thanet East": "East Thanet",
  "Worthing East and Shoreham": "East Worthing and Shoreham",
  "Aberdeenshire West and Kincardine": "West Aberdeenshire and Kincardine",
  "Wrekin, The": "The Wrekin",
  "Wiltshire South West": "South West Wiltshire",
  "Bedfordshire North": "North Bedfordshire",
  "Somerset North": "North Somerset",
  "Cotswolds South": "South Cotswolds",
  "Durham North": "North Durham",
  "Derbyshire North East": "North East Derbyshire",
  "Hampshire North West": "North West Hampshire",
  "Dorset West": "West Dorset",
  "Hull East": "Kingston upon Hull East",
  "Lancashire West": "West Lancashire",
  "Dorset South": "South Dorset",
  "Sussex Mid": "Mid Sussex",
  "Hertfordshire South West": "South West Hertfordshire",
  "Herefordshire North": "North Herefordshire",
  "Warwickshire North and Bedworth": "North Warwickshire and Bedworth",
  "Derbyshire South": "South Derbyshire",
  "Devon South": "South Devon",
  "Pembrokeshire Mid and South": "Mid and South Pembrokeshire",
  "Leicestershire North West": "North West Leicestershire",
  "Fife North East": "North East Fife",
  "Cornwall North": "North Cornwall",
  "Suffolk Central and North Ipswich": "Central Suffolk and North Ipswich",
  "Cornwall South East": "South East Cornwall",
  "Hull West and Haltemprice": "Kingston upon Hull West and Haltemprice",
  "Renfrewshire East": "East Renfrewshire",
  "Buckinghamshire Mid": "Mid Buckinghamshire",
  "Montgomeryshire and Glyndwr": "Montgomeryshire and Glyndŵr",
  "Dunbartonshire Mid": "Mid Dunbartonshire",
  "Hull North and Cottingham": "Kingston upon Hull North and Cottingham",
  "Hampshire East": "East Hampshire",
  "Wiltshire East": "East Wiltshire",
  "Devon South West": "South West Devon",
  "Norfolk Mid": "Mid Norfolk",
  "Norfolk North West": "North West Norfolk",
  "Norfolk South West": "South West Norfolk",
  "Essex North West": "North West Essex",
  "Bridlington and the Wolds": "Bridlington and The Wolds",
  "Suffolk West": "West Suffolk",
  "Cambridgeshire North West": "North West Cambridgeshire",
  "Suffolk South": "South Suffolk",
  "Northamptonshire South": "South Northamptonshire",
  "Somerset North East and Hanham": "North East Somerset and Hanham",
  "Dorset Mid and North Poole": "Mid Dorset and North Poole",
  "Northumberland North": "North Northumberland",
  "Devon Central": "Central Devon",
  "Leicestershire South": "South Leicestershire",
  "Ynys Mon": "Ynys Môn",
  "Basildon South and East Thurrock": "South Basildon and East Thurrock",
  "Derbyshire Mid": "Mid Derbyshire",
  "Cambridgeshire North East": "North East Cambridgeshire",
  "Durham, City of": "City of Durham",
  "Norfolk South": "South Norfolk",
  "Dunbartonshire West": "West Dunbartonshire",
  "Devon North": "North Devon",
  "Hertfordshire North East": "North East Hertfordshire",
  "Norfolk North": "North Norfolk",
  "Hampshire North East": "North East Hampshire"
}

df['name'] = df['name'].replace(lookups)

df = df.merge(hex_df, on='name', how='outer', indicator=True)
df = df.query("region != ''")
df = df[~df.region.isnull()]

df[['name', 'forecast', 'previousWinner', 'change', 'code']].to_csv('exit_poll_data_cleaned_probs.csv', index=False)

In [83]:
df

,name,forecast,previousWinner,change,code,region,_merge
0,Guildford,Liberal Democrat,None,True,E14001258,South East,both
1,Mitcham and Morden,Labour,None,True,E14001371,Greater London,both
2,Newcastle upon Tyne East and Wallsend,Labour,None,True,E14001378,North East,both
3,Scunthorpe,Labour,None,True,E14001462,Yorkshire and the Humber,both
4,Norwich North,Labour,None,True,E14001408,East of England,both
...,...,...,...,...,...,...,...
627,Richmond and Northallerton,Conservative,None,True,E14001444,Yorkshire and the Humber,both
628,Bootle,Labour,None,True,E14001113,North West,both
629,Poplar and Limehouse,Labour,None,True,E14001430,Greater London,both
630,Leeds North East,Labour,None,True,E14001321,Yorkshire and the Humber,both


In [76]:
df.iloc[-1].region

''

In [81]:
df[df.forecast.isnull()]

,name,forecast,previousWinner,change,code,region,_merge
170,Islington North,None,None,True,E14001305,Greater London,both


In [ ]:
hex_df.name